In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.font_manager
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from astroquery.simbad import Simbad
from scipy import stats
from scipy.stats import ks_2samp
from scipy.interpolate import interp1d
import matplotlib.ticker as mtick
from astropy.cosmology import FlatLambdaCDM
import copy
from adjustText import adjust_text
from astroquery.ned import Ned

import palettable.colorbrewer as pc
from cycler import cycler
plt.rcParams["axes.prop_cycle"]=cycler('color',pc.qualitative.Set3_12.mpl_colors)



Boorman_cosmo=FlatLambdaCDM(H0=67.3,Om0=0.315)
z_cut=0.044
%matplotlib inline



def def_dfs():
    parent=pd.read_excel("/Users/pboorman/Dropbox/data/dataframes/NuLANDS.xlsx","563")
    NuLANDS=parent.loc[parent.nulands==1.]
    HX=parent.loc[(parent.bat70_name_match==1.)|(parent.NuSflag==1.)&(parent.NuS_prop==0)]
    bat70=parent.loc[parent.bat70_name_match==1.]
    bat105=parent.loc[parent.bat105_name_match==1.]
    letter=parent.loc[(parent.bat70_name_match==1.)|
                       (parent.simbad=="2MASX J21391374-2646315")|
                       (parent.simbad=="IC 1198")]
    return parent,NuLANDS,HX,bat70,bat105,letter

II563,NuLANDS,HX,bat70,bat105,letter=def_dfs()

print("Total: "+str(len(II563)))
print("NuLANDS: "+str(len(NuLANDS)))
print("HX: "+str(len(HX)))
print("BAT70: "+str(len(bat70)))
print("BAT105: "+str(len(bat105)))
print("LETTER: "+str(len(letter)))




print(II563.groupby('Keel_type').count().II_ID)




Total: 563
NuLANDS: 84
HX: 105
BAT70: 46
BAT105: 52
LETTER: 48
Keel_type
BL-Lac            1
HII             122
HII?              2
LINER             9
LINER?            2
S1               80
S2              140
S2?               4
ZLOCAL          128
unclassified     75
Name: II_ID, dtype: int64


In [10]:
## nvss == NRAO VLA Sky Survey Catalog
## vlssr == VLA Low-Frequency Sky Survey Redux Source Catalog

import subprocess
import datetime

def make_fname(str_name):
    final_name=str_name.replace(" ","")
    return final_name
    
def get_table(instrument):
    instrument=instrument.lower()
    instrument_lookuptable={"nustar":"numaster",
                            "chandra":"chanmaster",
                            "suzaku":"suzamaster",
                            "swift":"swiftmastr",
                            "xmm":"xmmmaster",
                            "hst":"hstpaec",
                            "herschel":"herschllog",
                            "spitzer":"spitzmastr",
                            "nvss":"nvss",
                            "vlssr":"vlssr"}
    offset_lookup_arcmin={"nustar":"10",
                          "chandra":"21",
                          "suzaku":"10",
                          "swift":"25",
                          "xmm":"15",
                          "hst":"15",
                          "herschel":"1",
                          "spitzer":"3",
                          "nvss":"1",
                          "vlssr":"1"}
    
    return instrument_lookuptable[instrument],offset_lookup_arcmin[instrument]

def is_non_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0


## THIS PURELY RETURNS 1 IF OBSERVED, 0 IF NOT
def hquery(ID,name,coords,workingdir,instr="nustar"):
    SEARCH_STR=coords
    if coords.replace(" ","")=="":SEARCH_STR=name
    if SEARCH_STR.replace(" ","")=="":return np.nan
        
    hquery_perlpath="/Users/pboorman/Dropbox/data/0_analysis/heasarc_query/browse_extract_wget.pl"
    FNAME=make_fname(name)
    NOW=datetime.datetime.now().strftime("Y%Ym%md%d")
    INST=instr.lower()
    TABLE,OFFSET=get_table(INST) ## USING DEFAULT OFFSET FROM HEASARC
    if not os.path.isdir(workingdir+"/"+INST):os.mkdir(workingdir+"/"+INST)
    os.chdir(workingdir+"/"+INST)
    outfile="%(workingdir)s/%(INST)s/II_ID_%(ID)s_%(FNAME)s_hquery_%(INST)s_%(NOW)s.csv" %locals()
    heasarc_query="perl %(hquery_perlpath)s position='%(SEARCH_STR)s' radius=%(OFFSET)s table='%(TABLE)s' format=TEXT outfile=%(outfile)s" %locals()
    print("          Searching for %(INST)s obs. within %(OFFSET)s arcmin..." %locals())
    subprocess.call(heasarc_query,shell=True)
    try:
        result=pd.read_csv('%(outfile)s' %locals(),skiprows=4,delimiter='|')
        new_cols=[col for col in result.columns if not 'Unnamed' in col]
        result=result[new_cols]
        result.columns=[col.strip() for col in result.columns]
        if len(result)==0:
            os.remove(outfile)
            print("          0 OBSERVATIONS FOUND!")
            return 0
        else:
            result.to_csv('%(outfile)s' %locals())
            print("          "+str(len(result))+" OBSERVATIONS FOUND!")
        return 1
    except:
        os.remove(outfile)
        print("          0 OBSERVATIONS FOUND!")
        return 0
#         return result.obsid.values.astype(str)



In [11]:
II563,NuLANDS,HX,bat70,bat105,letter=def_dfs()
testbed=II563.loc[(II563.Keel_type!="ZLOCAL")]#(II563.pre_proposal_NuSflag==1)&(II563.pre563_BATflag==0)]
NTOT=len(testbed)
print(NTOT)
INSTRUMENT=["hst","herschel","spitzer","nustar","chandra","suzaku","xmm","nvss","vlssr"]#,"swift"
count=1
NOW=datetime.datetime.now().strftime("Y%Ym%md%d")
II563.loc[:,"nustar"+"_%(NOW)s" %locals()]=np.zeros(len(II563),dtype=int)
II563.loc[:,"chandra"+"_%(NOW)s" %locals()]=np.zeros(len(II563),dtype=int)
II563.loc[:,"suzaku"+"_%(NOW)s" %locals()]=np.zeros(len(II563),dtype=int)
II563.loc[:,"swift"+"_%(NOW)s" %locals()]=np.zeros(len(II563),dtype=int)
II563.loc[:,"xmm"+"_%(NOW)s" %locals()]=np.zeros(len(II563),dtype=int)

wrk_dir="/Users/pboorman/Dropbox/1_work/1c_nulands/0_analysis/HEASARC_QUERY/"
if not os.path.isdir(wrk_dir+"/"+NOW):
    os.mkdir(wrk_dir+"/"+NOW)
    os.chdir(wrk_dir+"/"+NOW)
    wrk_dir+="/"+NOW+"/"   
for i,src in testbed.iterrows():
    print("SEARCHING %(count)s/%(NTOT)s SOURCES" %locals())
    COORD_SEARCH=' '.join(src[["WISE_radeg","WISE_decdeg"]].astype(str))
    print(src.II_ID,src.NED,COORD_SEARCH)
    for INS in INSTRUMENT:
        print("     INSTRUMENT = %(INS)s" %locals())
        INS_OBSERVED=hquery(src.II_ID,src.NED,COORD_SEARCH,wrk_dir,INS)
        II563.loc[i,INS+"_%(NOW)s" %locals()]=INS_OBSERVED
    print("")
    count+=1

copy_II563=II563
cols=[I+"_"+NOW for I in INSTRUMENT]
cols.append("NED")
cols.append("simbad")
cols.append("II_ID")
cols.append("WISE_radeg")
cols.append("WISE_decdeg")
copy_II563[cols].to_csv("/Users/pboorman/Desktop/PREVIOUS_OBSERVATIONS_%(NOW)s.csv" %locals())

435
SEARCHING 1/435 SOURCES
562 2MASX J00005427+1830215 0.2262072 18.5059402
     INSTRUMENT = hst
          Searching for hst obs. within 15 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = herschel
          Searching for herschel obs. within 1 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = spitzer
          Searching for spitzer obs. within 3 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = nustar
          Searching for nustar obs. within 10 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = chandra
          Searching for chandra obs. within 21 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = suzaku
          Searching for suzaku obs. within 10 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = xmm
          Searching for xmm obs. within 15 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = nvss
          Searching for nvss obs. within 1 arcmin...
          0 OBSERVATIONS FOUND!
     INSTRUMENT = vlssr
      